Import the packages and load the XLSX file with Pandas

In [111]:
import pandas as pd
import sqlite3
from datetime import datetime

file_path = "/Users/joshlopez/PyCharmMiscProject/NP_task_eng_revised.xlsx"
df1 = pd.read_excel(file_path, sheet_name='Data_1')
df2 = pd.read_excel(file_path, sheet_name='Data_2')


1.  inspection data types

In [112]:
print(df1.dtypes)

PREFIX                    object
country_code              object
region_id                 object
first_activity            object
last_activity     datetime64[ns]
dtype: object


2. Convert to correct data types

In [113]:
df1["first_activity"] = pd.to_datetime(df1["first_activity"], errors='coerce')
df1['region_id'] = pd.to_numeric(df1['region_id'], errors='coerce')
print(df1.dtypes)

PREFIX                    object
country_code              object
region_id                float64
first_activity    datetime64[ns]
last_activity     datetime64[ns]
dtype: object


3. Connect to SQLite3

In [114]:
conn = sqlite3.connect("my_database.db")
df1.to_sql("Data_1", conn, index=False, if_exists='replace')
current_year = datetime.now().year

4. Create a new tables with anomaly flags

In [115]:
query = """
SELECT *,
CASE
  WHEN PREFIX IS NULL AND CAST(strftime('%Y',first_activity) AS INTEGER) > 2025 THEN 'Future Dates + Missing PREFIX'
  WHEN PREFIX IS NULL THEN 'Missing PREFIX'
  WHEN PREFIX IN (SELECT PREFIX from Data_1 GROUP BY PREFIX HAVING count(*)>1) THEN 'Duplicate PREFIX'
  WHEN last_activity < first_activity THEN 'Invalid Dates'
  WHEN LENGTH(country_code) <> 2 THEN 'Invalid Country'
  WHEN region_id IS NULL AND first_activity IS NULL THEN 'Missing date + Invalid Region'
  WHEN region_id <=0 OR region_id IS NULL THEN 'Invalid Region'
  ELSE 'OK'
  END AS issue
FROM Data_1
"""

df_with_issues = pd.read_sql(query, conn)

1.1 Anomalies from the Data_1 table

The approach used was:
- Identify the data types of each column.
- Identify missing or null values.
- Set rules for outliers, for instance dates in the future, duplicated values, and length of the value out of the expected range.
- Create a column displaying the anomalies to be shared with the team in order to fix the issues.

In [116]:
issues_only = df_with_issues[df_with_issues["issue"] != 'OK']
print(issues_only)

                      PREFIX country_code  region_id       first_activity  \
159     14fd695f545x537ab749           PL       23.0  2015-09-27 13:55:12   
169     14fd695f545x537ab749           PL       11.0  2015-10-07 15:58:21   
728     14fd7a9d90ex1626dc75           RO        3.0  2015-11-20 10:21:01   
844     14fd7a9d90ex1626dc75           PL        4.0  2015-09-21 18:00:58   
2068  14fe06679c7x375369ds4c           PL        NaN                 None   
2240    14fd6247ce1x19dc5f27           PL       13.0  2015-09-29 23:30:47   
2276    14fd6247ce1x19dc5f27           PL        1.0  2015-09-19 20:39:39   
2353                    None           PL       23.0  2040-12-20 07:39:42   

            last_activity                          issue  
159   2015-09-27 13:55:12               Duplicate PREFIX  
169   2015-10-07 15:58:21               Duplicate PREFIX  
728   2015-11-20 10:21:01               Duplicate PREFIX  
844   2015-09-21 18:00:58               Duplicate PREFIX  
2068  2015-

1.2 Quantitative and percentage distribution of user by region

The most represented region in this data is region_id 2.0 with 13.67% of users
in contrast the least represented region is region_id 23.0 with 0.04% of users.

In [117]:
query2 = """
SELECT region_id,
count(*) AS user_count,
ROUND(count(*)*100.0/(select count(*) from Data_1),2) AS percentage
FROM Data_1
WHERE region_id IS NOT NULL
GROUP BY region_id
ORDER BY percentage desc
"""

df_2_tasks = pd.read_sql(query2, conn)
print(df_2_tasks)

    region_id  user_count  percentage
0         2.0         684       13.67
1         6.0         575       11.49
2         1.0         573       11.45
3         3.0         388        7.75
4         4.0         366        7.31
5         7.0         341        6.81
6         5.0         332        6.63
7        15.0         303        6.05
8         8.0         280        5.59
9        11.0         236        4.72
10       17.0         197        3.94
11       14.0         181        3.62
12       13.0         168        3.36
13        9.0         152        3.04
14       18.0         128        2.56
15       12.0          98        1.96
16       23.0           2        0.04


1.3 Percentage of user who only did one activity and never came back

The percentage of user who did only one activity is 44.06%

In [118]:
# SQL approach
query3 = """
SELECT ROUND((SELECT count(*) FROM Data_1 WHERE first_activity=last_activity)*100.0
/ (SELECT count(*) FROM Data_1 WHERE first_activity IS NOT NULL AND last_activity IS NOT NULL),2) AS perc_unique_users
"""
df_3_tasks = pd.read_sql(query3, conn)
print(df_3_tasks)


   perc_unique_users
0              44.06


1.4 List of customers that did only 1 activity

The way of identifying the customers who did only one activity is by comparing the columns first_activity and last_activity for prefix with the same values in both columns

The answer is a total of 2205 unique customers.

In [119]:
query4 = """
SELECT PREFIX FROM Data_1 WHERE first_activity=last_activity
"""
df_4_tasks = pd.read_sql(query4, conn)
print(df_4_tasks)

                    PREFIX
0     125e7c32c75x1470d378
1     137178d76e7x697d40d9
2     13951d78ccax69d403c1
3     14fc1cfc985x33fe98a6
4     14fd4c69808x72b705a7
...                    ...
2200  14fd59804f0x10529bc8
2201  14fe196266fx621c950d
2202   14fe19e2cffxded6c21
2203  14fe1ae473ax5ae7e122
2204   14fe1b040e1xa171172

[2205 rows x 1 columns]


1.5 Actions suggested to activate users who did only one activity

I will provide the specific list of users and details to the Marketing team and also managers in charge of customer retention and I would advise them to set a customized marketing campaign for these users with attractive coupons or discounts for accessing the site once again and purchasing the first item, the campaign can be also target the concentration of users in certain regions.